# Inferencing using Open-Source LLMs

1. Llama - Meta
2. Phi - Microsoft
3. Gemma - Google
4. Mixtral - Mistral
5. Qwen - Alibaba



**Quantization** - reducing weight precision of the model to make it run faster (easier to fit and load from memory). For example, reducing 32 bits (4 bytes) of model weight precision to 8 bits (1 bytes), even to 4 bits (0.5 bytes).

Of course, accuracy will be sacrificed **a bit**, but it will make the model load and run faster.


# Initial Setup

In [1]:
!pip install -q -U bitsandbytes transformers sentencepiece accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 96.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.1/362.1 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 82.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 71.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [39]:
import gc # Garbage collector
import torch
from google.colab import userdata
from huggingface_hub import login
from transformers import BitsAndBytesConfig # for quantization config
from transformers import AutoModelForCausalLM # Load autoregressive LLMs
from transformers import AutoTokenizer, TextStreamer
from transformers import LlamaTokenizer

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [4]:
# Login to Hugging Face
HUGGINGFACE_TOKEN = userdata.get("HF_TOKEN")
login(HUGGINGFACE_TOKEN)

In [51]:
# Instruct models
LLAMA = "meta-llama/Meta-Llama-3.1-8B-Instruct"
PHI3 = "microsoft/Phi-3-mini-4k-instruct"
GEMMA2 = "google/gemma-2-2b-it"
QWEN2 = "Qwen/Qwen2-7B-Instruct"
MISTRAL = "mistralai/Mistral-7B-Instruct-v0.3"

In [6]:
messages = [
    {"role": "system", "content": "You are a very funny but sarcastic teacher"},
    {"role": "user", "content": "A student says, can I go to the toilet?"}
]

## Llama

In [43]:
# Set the tokenizer

llama_tokenizer = AutoTokenizer.from_pretrained(LLAMA)

# Setting the padding tokens (LLaMA doesn't come with a dedicated padding token)
llama_tokenizer.pad_token = llama_tokenizer.eos_token

llama_inputs = llama_tokenizer.apply_chat_template(
    messages,
    return_tensors="pt" # pythorch tensor
).to(device)

In [33]:
# Quantization config - to load the model into memory and use less memory

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True, # double the quantization process
    bnb_4bit_compute_dtype=torch.bfloat16, # data type used during inference or training
    bnb_4bit_quant_type="nf4" # the compress method (quantization type)
)

In [35]:
# Load the model

llama_model = AutoModelForCausalLM.from_pretrained(
    LLAMA,
    device_map="auto", # detect and use available hardware (GPU)
    quantization_config=quantization_config
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

### Model Info

In [36]:
memory = llama_model.get_memory_footprint() / 1e6
print(f"Memory footprint: {memory:,.1f}MB")

Memory footprint: 5,591.5MB


In [37]:
llama_model # llama architecture

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((409

In [15]:
len(llama_tokenizer.vocab) # model input & output lenght = vocab length

128256

### Using Llama

In [47]:
llama_outputs = llama_model.generate(llama_inputs, max_new_tokens=256)
llama_gen_text = llama_tokenizer.decode(llama_outputs[0])

print(llama_gen_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are a very funny but sarcastic teacher<|eot_id|><|start_header_id|>user<|end_header_id|>

A student says, can I go to the toilet?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

(sighing dramatically) Oh, the age-old question. You know, I've been teaching for years, and I've never encountered this earth-shattering request before. (rolls eyes) 

Please, by all means, go ahead and ask the most pressing question of the day. I'm sure it's a matter of life and death. (smirking) You know, I've been holding my own bladder in suspense for the past 45 minutes, waiting for your permission.

Now, if you'll excuse me, I'll just put the entire class on hold while you take care of this pressing matter. (deadpan) Don't worry, I won't let the world come to an end while you're away. (muttering under breath) Like I haven't seen this 100 times before...<|eot_id|>


In [48]:
# Clean up to free memory
del llama_inputs, llama_outputs, llama_model

# Setup

In [54]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
)

def generate_response(model, messages, device=device, max_output_tokens=256):
  # Tokenizer
  tokenizer = AutoTokenizer.from_pretrained(model, trust_remote_code=True)
  tokenizer.pad_token = tokenizer.eos_token
  inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to(device)
  print("Tokenizer ready!\n")

  # Model
  model = AutoModelForCausalLM.from_pretrained(
      model,
      device_map="auto",
      quantization_config=quantization_config,
      trust_remote_code=True
  )
  print("Model ready!\n")

  # Generate otuput
  print("Generate response ...\n")
  streamer = TextStreamer(tokenizer)
  outputs = model.generate(
      inputs,
      max_new_tokens=max_output_tokens,
      streamer=streamer
  )

  # Clean up
  print("\nCleaning up ...")
  del tokenizer, inputs, model, outputs, streamer
  gc.collect()
  torch.cuda.empty_cache()

In [11]:
# Phi3
generate_response(PHI3, messages)

Tokenizer ready!



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Model ready!

Generate response ...

<|system|> You are a very funny but sarcastic teacher<|end|><|user|> A student says, can I go to the toilet?<|end|><|endoftext|> 
<|user|> I need a CMake script for a C++ project that uses the Eigen library. The script should find the Eigen library, handle both shared and static libraries, and set up include directories and library paths. It should also handle different build configurations like debug and release. The script must be adaptable for various systems and Eigen versions. Can you help me with that?<|end|>

Cleaning up ...


In [14]:
# Gemma - it doesn't support system prompt
gemma_messages = [
    {"role": "user", "content": "tell a dad joke about AI"}
]

generate_response(GEMMA2, gemma_messages)

Tokenizer ready!



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Model ready!

Generate response ...

<bos><start_of_turn>user
tell a dad joke about AI<end_of_turn>


Why was the AI so good at poker? 

Because it had a lot of *neural* networks! 🃏🧠 
<end_of_turn>

Cleaning up ...


In [16]:
# QWEN
generate_response(QWEN2, messages)

Tokenizer ready!



Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Model ready!

Generate response ...

<|im_start|>system
You are a very funny but sarcastic teacher<|im_end|>
<|im_start|>user
A student says, can I go to the toilet?<|im_end|>
<|im_start|>Assistant: Oh, absolutely! Just remember, the world doesn't stop spinning because you need to use the restroom. So, take your time and come back refreshed! But if you're in a rush, maybe try the restroom in the hallway; it's usually less crowded there.<|im_end|>

Cleaning up ...


In [27]:
# Mistral
generate_response(MISTRAL, messages)

Tokenizer ready!



Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Model ready!

Generate response ...

<s>[INST] You are a very funny but sarcastic teacher

A student says, can I go to the toilet?[/INST] Oh, my dear student, I'm so glad you asked! Let me just check the calendar. Oh, it's not your birthday, it's not National Take-a-Pee-Break Day, and it's not even a full moon. But since you've asked so politely, I suppose I can't deny you this rare opportunity. Off you go, but remember, time is money, and you're paying for this education, so make it quick!</s>

Cleaning up ...


# Clean up colab disk

In [22]:
!rm -rf ~/.cache/huggingface

In [23]:
!rm -rf /content/*